In [1]:
import torch
from torch import nn
import matplotlib.pyplot as plt
import numpy as np

from config import Config

In [2]:
# Set up the device
config = Config()
print('Current device:', config.device)

Current device: mps


In [3]:
# Load tensors. The map_location argument is used to load the tensors onto the specified device.
train_data = torch.load('temp/data/train_data.pth', map_location=config.device)
test_data = torch.load('temp/data/test_data.pth', map_location=config.device)

### Construir un modelo

- *Todos nuestros modelos en PyTorch van a heredar sus funcionalidades de la clase `nn.Module`. Esta es la clase base que nos aporta todas las funcionalidades necesarias para construir y entrenar una red neuronal.*
- *Algunas de estas funcionalidades son:*
    - *La capacidad de hacer un seguimiento de todos los parámetros del modelo que fueron definidos como atributos (esto quiere decir que se definieron usando la clase `nn.Parameter`<sup>1</sup>).*
        - *Entonces, si definimos un atributo utilizando la clase `nn.Parameter`, PyTorch va a automáticamente registrar ese atributo y lo va a tener en cuenta al momento de computar los gradientes o al mover el modelo de un dipositivo a otro (e.g., de CPU a GPU).*
        - *Podemos acceder a estos parámetros utilizando el método `parameters()` o `state_dict()` (recomendado).*
    - *La posibilidad de crear arquitecturas complejas, al poder definir módulos dentro de otros.*

<sup>1</sup> *`nn.Parameter` define una clase especial de tensores que se registran de manera automática cuando se definen dentro de una instancia de `nn.Module`. Muchas veces, cuando definamos las capas ocultas de nuestras redes neuronales, vamos a utilizar clases pre-definidas como `nn.Linear` o `nn.Conv2D`, las cuales definen estos atributos por nosotros, por lo que no vamos a tener que llamar a la función `nn.Parameter` explicitamente.*

In [4]:
class LinearRegressionModel(nn.Module):
    def __init__(self, device: torch.device = torch.device('cpu')):
        '''
        Initializes a LinearRegressionModel.
        
        Args:
            device (torch.device): Device to run the model on. Defaults to CPU.
        '''
        super().__init__()
        self.weight = nn.Parameter(torch.randn(1, dtype=torch.float32))
        self.bias = nn.Parameter(torch.randn(1, dtype=torch.float32))
        
        self.to(device=device)
    
    def forward(self, X: torch.Tensor) -> torch.Tensor:
        return self.weight * X + self.bias

- *Algunas consideraciones:*
    - *Al definir el constructor de la clase (i.e., al definir la función `__init__`), lo primero que agregamos es la línea `super().__init__()`, la cual inicializa a la clase padre. Si no la incluímos, la clase no va a poder heredar las funcionalidades de `nn.Module` que necesitamos para entrenar los modelos.*
    - *Cuando se crea una clase a partir del módulo `nn.Module`, es necesario implementar el método `forward()`. Este método es muy importante porque define el grafo computacional de nuestro modelo (i.e., como fluye la información a lo largo de la red neuronal), el cual utilizamos para calcular los gradientes.*
    - *Por defecto, PyTorch inicializa cualquier módulo en la CPU, independientemente del dispositivo que se encuentre disponible. Incluímos la línea `self.to(device=device)` al final del constructor de nuestra clase para automáticamente mover nuestro modelo (junto con todos sus parámetros/buffers) al dispositivo deseado.*

In [5]:
# Create a random seed
torch.manual_seed(42)

# Create a model instance
model_0 = LinearRegressionModel(device=config.device)

# Check the model's parameters
# We can either use the `.parameters()` method or the `state_dict()` method
model_0.state_dict()  # returns a dictionary of the model's parameters

OrderedDict([('weight', tensor([0.3367], device='mps:0')),
             ('bias', tensor([0.1288], device='mps:0'))])